# 📊 Анализ данных терразитовой штукатурки

Этот ноутбук анализирует Excel файл с рецептами терразитовой штукатурки.


In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Библиотеки загружены")

## 1. Загрузка данных из Excel

In [ ]:
# Путь к файлу
excel_path = Path('../data/raw/Рецептуры терразит.xlsx')
print(f"📂 Путь к файлу: {excel_path}")
print(f"📂 Файл существует: {excel_path.exists()}")

if excel_path.exists():
    # Загружаем Excel
    df = pd.read_excel(excel_path, sheet_name=0)
    print(f"✅ Файл загружен: {df.shape[0]} строк, {df.shape[1]} столбцов")
else:
    print("❌ Файл не найден! Проверьте путь.")

## 2. Предварительный просмотр данных

In [ ]:
if 'df' in locals():
    # Показать первые строки
    print("📋 Первые 5 строк:")
    display(df.head())
    
    # Показать информацию о данных
    print("\n📊 Информация о данных:")
    print(df.info())
    
    # Показать названия столбцов
    print("\n📝 Названия столбцов:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")

## 3. Анализ структуры данных

In [ ]:
if 'df' in locals():
    # Найдем столбец с названиями рецептов
    recipe_col = None
    for col in df.columns:
        if 'наименование' in str(col).lower() or 'рецепт' in str(col).lower():
            recipe_col = col
            break
    
    print(f"📌 Столбец с названиями рецептов: {recipe_col}")
    
    if recipe_col:
        # Сколько всего рецептов?
        recipes = df[recipe_col].dropna().unique()
        print(f"\n🧪 Всего уникальных рецептов: {len(recipes)}")
        
        # Показать первые 10 рецептов
        print("\n📋 Примеры рецептов:")
        for i, recipe in enumerate(recipes[:10], 1):
            print(f"  {i}. {recipe}")
    
    # Статистика по числовым столбцам
    print("\n📈 Статистика по числовым данным:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        display(df[numeric_cols].describe())
    else:
        print("❌ Числовые столбцы не найдены")

## 4. Визуализация данных

In [ ]:
if 'df' in locals() and recipe_col:
    # Создадим графики
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Распределение типов рецептов
    recipe_types = []
    for recipe in df[recipe_col].dropna():
        if 'терразит' in str(recipe).lower():
            recipe_types.append('Терразит')
        elif 'шовный' in str(recipe).lower():
            recipe_types.append('Шовный')
        else:
            recipe_types.append('Другой')
    
    type_counts = pd.Series(recipe_types).value_counts()
    axes[0, 0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Распределение типов рецептов')
    
    # 2. Топ-10 самых частых компонентов (по наличию в рецептах)
    component_presence = {}
    for col in df.columns:
        if col != recipe_col:
            # Сколько рецептов содержат этот компонент?
            count = (df[col].fillna(0) > 0).sum()
            if count > 0:
                component_presence[col] = count
    
    # Сортируем и берем топ-10
    top_components = sorted(component_presence.items(), key=lambda x: x[1], reverse=True)[:10]
    components_names = [c[0][:20] + '...' if len(c[0]) > 20 else c[0] for c in top_components]
    components_counts = [c[1] for c in top_components]
    
    axes[0, 1].barh(components_names, components_counts)
    axes[0, 1].set_xlabel('Количество рецептов')
    axes[0, 1].set_title('Топ-10 компонентов по частоте использования')
    axes[0, 1].invert_yaxis()
    
    # 3. Распределение общего веса компонентов
    if len(numeric_cols) > 0:
        total_weights = df[numeric_cols].sum(axis=1)
        axes[1, 0].hist(total_weights, bins=20, edgecolor='black')
        axes[1, 0].set_xlabel('Общий вес (кг)')
        axes[1, 0].set_ylabel('Количество рецептов')
        axes[1, 0].set_title('Распределение общего веса рецептов')
    
    # 4. Тепловая карта корреляций
    if len(numeric_cols) > 1:
        corr_matrix = df[numeric_cols].corr()
        im = axes[1, 1].imshow(corr_matrix, cmap='coolwarm', aspect='auto')
        axes[1, 1].set_title('Корреляция между компонентами')
        plt.colorbar(im, ax=axes[1, 1])
    
    plt.tight_layout()
    plt.show()
    
    print("📈 Графики созданы успешно!")

## 5. Запуск основного скрипта обработки

In [ ]:
# Запускаем наш основной скрипт обработки
print("🔄 Запуск основного скрипта обработки...")

# Добавляем путь к скриптам
sys.path.append('../scripts')

try:
    from process_excel import ExcelProcessor, main
    
    # Запускаем обработку
    result = main()
    
    if result:
        print("\n✅ Обработка завершена успешно!")
        print(f"📄 JSON файл создан: {result['json_path']}")
        print(f"📊 Графики сохранены: {result['viz_path']}")
    else:
        print("❌ Обработка завершилась с ошибкой")
except Exception as e:
    print(f"❌ Ошибка при запуске скрипта: {e}")

## 6. Проверка обработанных данных

In [ ]:
# Проверим созданные файлы
processed_json = Path('../data/processed/recipes.json')
if processed_json.exists():
    print("✅ Файл recipes.json найден")
    
    # Загрузим и покажем структуру
    with open(processed_json, 'r', encoding='utf-8') as f:
        recipes_data = json.load(f)
    
    print(f"📊 Загружено рецептов: {len(recipes_data)}")
    
    # Покажем первый рецепт для примера
    if recipes_data:
        first_recipe = recipes_data[0]
        print(f"\n🧪 Пример рецепта:")
        print(f"   ID: {first_recipe['id']}")
        print(f"   Название: {first_recipe['name']}")
        print(f"   Тип: {first_recipe['type']}")
        
        # Топ-5 компонентов по весу
        sorted_comps = sorted(first_recipe['components'].items(), 
                             key=lambda x: x[1], reverse=True)[:5]
        print(f"   Топ-5 компонентов:")
        for comp, weight in sorted_comps:
            if weight > 0:
                print(f"     • {comp[:30]}: {weight} кг")
else:
    print("❌ Файл recipes.json не найден")

## Выводы

1. Данные успешно загружены и проанализированы
2. Создана структурированная JSON версия рецептов
3. Подготовлены визуализации для анализа
4. Данные готовы для обучения ML моделей